In [ ]:
import sys
!rm -rf thesis
!git clone --branch master https://github.com/mattalejo/thesis

Cloning into 'thesis'...
remote: Enumerating objects: 1518, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 1518 (delta 170), reused 85 (delta 59), pack-reused 1267 (from 1)
Receiving objects: 100% (1518/1518), 215.67 MiB | 21.24 MiB/s, done.
Resolving deltas: 100% (564/564), done.
Updating files: 100% (306/306), done.


In [ ]:
sys.path.append("/content/thesis/")

In [ ]:
import yfinance as yf
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import math as m

from sklearn.preprocessing import StandardScaler
# from tensorboard_logger import configure

import copy
import sys
import os

import prep_data
import train

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader, TensorDataset

from models.transformer import Transformer
from models.rnn import RNN, RNN5Day
from models.transformer_encoder import TransformerEncoder
from models.cnn import CNN
from models.lstm import LSTM

# import mpu.io
# import json

In [1]:
seed = 1
torch.manual_seed(seed)

NameError: name 'torch' is not defined

In [ ]:
if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "cpu"
device = torch.device(dev)

print(dev)

cpu


In [ ]:
BATCH_SIZE = [512]
DROPOUT = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
HORIZON = [1, 5]
PERIODS = {
    "train": ("2010-01-01", "2017-12-31"),
    "test": ("2018-01-01", "2019-12-31"),
    "gfc":("2008-01-01", "2008-12-31"),
    "covid":("2020-01-01", "2022-12-31")
 }

for horizon in HORIZON:
  for batch_size in BATCH_SIZE:
    for dropout in DROPOUT:
      MODELS = [CNN(dropout=dropout), RNN(dropout=dropout), LSTM(dropout=dropout)]
      for models in MODELS:
          model, best_model, loss, train_pred, test_pred, _ = train.train(
              model=models,
              seq_len=128,
              horizon=horizon,
              max_epoch=1000,
              batch_size=batch_size,
              loss=nn.MSELoss(),
              optimizer=optim.Adam,
              lr=1e-4,
              device=dev
          )
          config = f"updated_{model.model_type}_dropout-{dropout}_seed-{seed}"
          os.mkdir(config)

          torch.save(model, f"model_{config}.pth")
          torch.save(best_model, f"best_model_{config}.pth")
          loss.to_csv(f"metadata_{config}.csv")
          mpu.io.write(f"train_{config}.json", jsonify(train_pred))
          mpu.io.write(f"test_{config}.json", jsonify(test_pred))

          for periods, range in PERIODS.items():
            start_date, end_date = range
            date_range = pd.date_range(start_date, end_date)
            df, loss = train.test(
              model=model,
              seq_len=128,
              horizon=horizon,
              batch_size=0,
              start_date=start_date,
              end_date=end_date,
              loss=nn.MSELoss(),
              lr=1e-4,
              scale_method="std",
              device=dev
            )

            if horizon == 1:
              df = pd.DataFrame({"y": df["y"], "pred": df["pred"]},  index=date_range)

            if horizon != 1:
              df["index"] = date_range
              df["yy"] = np.array([i[4] for i in df["y"]])
              df["5-day"] = np.array([i[4] for i in df["pred"]])
              # print(df)
              # print(df["yy"].shape, df["pred"].shape)

              # print(df["5-day"])

              df = pd.DataFrame({"y" : df["yy"], "pred": df["5-day"]}, index=df["index"])

          os.chdir("..")



[*********************100%***********************]  1 of 1 completed
/content/thesis/prep_data.py:28: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


num_params: 258627
Epoch 000 | train loss 0.9804 | test_loss 0.6460 | wall_time 1.3067 | process_time 1.3140
Epoch 001 | train loss 0.5555 | test_loss 0.5569 | wall_time 1.0829 | process_time 1.0816
Epoch 002 | train loss 0.5962 | test_loss 0.5316 | wall_time 1.0750 | process_time 1.0836
Epoch 003 | train loss 0.5379 | test_loss 0.5212 | wall_time 1.1062 | process_time 1.1144


KeyboardInterrupt: 